In [ ]:
#Many of these prep functions have been found around the web and/or borrowed from https://nbviewer.jupyter.org/url/norvig.com/ipython/Advent%20of%20Code.ipynb

# Python 3.x
import re
import numpy as np
import math
import urllib.request

from collections import Counter, defaultdict, namedtuple, deque
from functools   import lru_cache
from itertools   import permutations, combinations, chain, cycle, product, islice
from heapq       import heappop, heappush

def Input(day):
    "Open this day's input file."
    filename = '/home/isaac/AdventOfCode2018/day{}.txt'.format(day)
    return open(filename)
    
def input_lines(day):
    fobj = Input(day)
    lines = fobj.readlines()
    fobj.close()
    return lines

def transpose(matrix): return zip(*matrix)

def first(iterable): return next(iter(iterable))

def nth(iterable, n, default=None):
    "Returns the nth item of iterable, or a default value"
    return next(islice(iterable, n, None), default)

cat = ''.join

Ø   = frozenset() # Empty set
inf = float('inf')
BIG = 10 ** 999

def grep(pattern, lines):
    "Print lines that match pattern."
    for line in lines:
        if re.search(pattern, line):
            print(line)
            
def gengrep(pattern, lines):
    patc = re.compile(pattern)
    return (line for line in lines if patc.search(line))

def groupby(iterable, key=lambda it: it):
    "Return a dic whose keys are key(it) and whose values are all the elements of iterable with that key."
    dic = defaultdict(list)
    for it in iterable:
        dic[key(it)].append(it)
    return dic

def powerset(iterable):
    "Yield all subsets of items."
    items = list(iterable)
    for r in range(len(items)+1):
        for c in combinations(items, r):
            yield c

# 2-D points implemented using (x, y) tuples
def X(point): return point[0]
def Y(point): return point[1]

def neighbors4(point): 
    "The four neighbors (without diagonals)."
    x, y = point
    return ((x+1, y), (x-1, y), (x, y+1), (x, y-1))

def neighbors8(point): 
    "The eight neighbors (with diagonals)."
    x, y = point 
    return ((x+1, y), (x-1, y), (x, y+1), (x, y-1),
            (X+1, y+1), (x-1, y-1), (x+1, y-1), (x-1, y+1))

def manhattan_distance(p, q=(0, 0)): 
    "City block distance between two points."
    return abs(X(p) - X(q)) + abs(Y(p) - Y(q))

def euclidean_distance(p, q=(0, 0)): 
    "Euclidean (hypotenuse) distance between two points."
    return math.hypot(X(p) - X(q), Y(p) - Y(q))

def trace1(f):
    "Print a trace of the input and output of a function on one line."
    def traced_f(*args):
        result = f(*args)
        print('{}({}) = {}'.format(f.__name__, ', '.join(map(str, args)), result))
        return result
    return traced_f

def astar_search(start, h_func, moves_func):
    "Find a shortest sequence of states from start to a goal state (a state s with h_func(s) == 0)."
    frontier  = [(h_func(start), start)] # A priority queue, ordered by path length, f = g + h
    previous  = {start: None}  # start state has no previous state; other states will
    path_cost = {start: 0}     # The cost of the best path to a state.
    while frontier:
        (f, s) = heappop(frontier)
        if h_func(s) == 0:
            return Path(previous, s)
        for s2 in moves_func(s):
            new_cost = path_cost[s] + 1
            if s2 not in path_cost or new_cost < path_cost[s2]:
                heappush(frontier, (new_cost + h_func(s2), s2))
                path_cost[s2] = new_cost
                previous[s2] = s
    return dict(fail=True, front=len(frontier), prev=len(previous))
                
def Path(previous, s): 
    "Return a list of states that lead to state s, according to the previous dict."
    return ([] if (s is None) else Path(previous, previous[s]) + [s])

In [ ]:
#day1 part 1 
#Simply sum all the lines of the input
sum((int(l.strip()) for l in input_lines(1)))

In [ ]:
#day1 part2
#Do the same as in part 1, but keep a running list of intermediate values
#find the first value to be reached twice
input_generator = cycle(int(l.strip()) for l in input_lines(1))
visited = [0]
for iv in input_generator:
    nxt = visited[-1]+iv
    if nxt in visited:
        break
    visited.append(nxt)
print(nxt)
print(visited[-1])

In [ ]:
#day2 part1
#Count the number of strings that have any letter appearing exactly three times
#Count the number of strings that have any letter appearing exactly two times
#multiply those numbers to get answer
lines = input_lines(2)
g1 = (Counter(l) for l in lines)
g2 = (Counter(l) for l in lines)
threes = sum(1 for x in g1 if 3 in x.values())
twos = sum(1 for x in g2 if 2 in x.values())
threes*twos

In [ ]:
#day2 part2
#find the ids that differ by only one letter in the same position
lines = input_lines(2)
def nletterdiff(s1, s2):
    return sum(1 for l1,l2 in zip(s1,s2) if l1!=l2)
def find_nearest_match(lns):
    for c in combinations(lns, 2):
        if nletterdiff(*c)==1:
            return c
def remove_differing_letter(s1,s2):
     return cat(l1 for l1,l2 in zip(s1,s2) if l1==l2)
print(remove_differing_letter(*find_nearest_match(lines)))
            

In [ ]:
#day3 part1
#find number of square inches in rectangles that are claimed by more than one sub-rectangle
lines = input_lines(3)
def process_line(line):
    id, _, a, b = line.strip().split()
    min_x, min_y = map(int, a.strip(':').split(','))
    sz_x, sz_y = map(int, b.split('x'))
    return id, min_x, min_y, min_x+sz_x, min_y+sz_y
def find_max_dimensions(lns):
    maxx = 0
    maxy = 0
    for line in lns:
        *_, max_x, max_y = process_line(line)
        maxx = max_x if max_x>maxx else maxx
        maxy = max_y if max_y>maxy else maxy
    return maxx, maxy
def claim_matrix(lns):
    fabric = np.zeros(find_max_dimensions(lns), dtype='int')
    for id, min_x, min_y, max_x, max_y in map(process_line, lns):
        fabric[min_x:max_x, min_y:max_y]+=1
    return fabric
def n_overlap(lns):
    mtx = claim_matrix(lns)
    return np.sum(mtx>1)
n_overlap(lines)

In [ ]:
#day3 part2
#Find the claim that does not overlap any others
#Can do only two iterations by building the claim matrix, then checking each claim against the matrix
lines = input_lines(3)
def is_unique(sub_matrix):
    return np.product(sub_matrix.shape) == np.sum(sub_matrix)
def find_unique_claim(lns):
    mtx = claim_matrix(lns)
    for id, min_x, min_y, max_x, max_y in map(process_line, lns):
        if is_unique(mtx[min_x:max_x, min_y:max_y]):
            return id
find_unique_claim(lines)

In [ ]:
#day4 part1
#Sort Entries
#Find guard with most minutes slept, and minute with most sleep.
lines4 = input_lines(4)
import datetime
def process_lines(lns):
    fmtstring = '%Y-%m-%d %H:%M'
    return sorted([(datetime.datetime.strptime(l.split(']')[0].strip('['), fmtstring), 
                    l.split(']')[1].strip().rstrip()) for l in lns],
                  key=lambda x: x[0])
process_lines(lines4)

def guard_records(lns):
    sorted_lines = process_lines(lns)
    guards = defaultdict(list)
    current_guard = 0
    for time, event in sorted_lines:
        if 'begins shift' in event:
            current_guard = int(event.split()[1].strip('#'))
        guards[current_guard].append((time, event))
    return guards
            
def minutes_asleep_counter(guard_rec):
    counter = np.zeros(60)
    awake = True
    asleep=0
    for time, event in guard_rec:
        if 'begins shift' in event:
            awake=True
            asleep = 0
        elif event=='falls asleep':
            awake=False
            asleep = time.minute
        elif event == 'wakes up':
            awake=True
            counter[asleep:time.minute] += 1
    return counter

def guard_counters(lns):
    records = guard_records(lns)
    #grds = {}
    max_asleep = 0
    max_minute = 0
    max_guard = 0
    for guard, record in records.items():
        ctr = minutes_asleep_counter(record)
        asleep = np.sum(ctr)
        if asleep > max_asleep:
            max_asleep = asleep
            max_minute = np.argmax(ctr)
            max_guard = guard
        #grds[guard] = {'asleep':asleep, 'minute':np.argmax(ctr)}
    return max_guard, max_asleep, max_minute, max_guard*max_minute
guard_counters(lines4)

In [ ]:
#day4 part2
def guard_counters2(lns):
    records = guard_records(lns)
    grds = {}
    for guard, record in records.items():
        grds[guard] = minutes_asleep_counter(record)
    return grds

def strategy2(counter_dict):
    guards, minute_counters = list(zip(*counter_dict.items()))
    minute_matrix = np.array(minute_counters)
    
    #return guards, minute_matrix.shape, np.max(minute_matrix, axis=1), np.argwhere(minute_matrix>=np.max(minute_matrix))
    gd, min = tuple(np.argwhere(minute_matrix>=np.max(minute_matrix))[0])
    return guards[gd]*min
ctrs = guard_counters2(lines4)
print(strategy2(ctrs))

In [ ]:
#day5 part1:
#process polymer(string)
#rules:
#If two adjacent letters are equal with opposite polarization, they annhilate
#iterate with this rule until no more pairs of letters with equal and opposite polarization exist

def find_all_pairs(strng):
    pair_locs = []
    last_ch = ''
    for i, ch in enumerate(strng):
        if ch.upper()==last_ch.upper() and last_ch.isupper()+ch.isupper()==1:
            if len(pair_locs)==0 or i != pair_locs[-1]+1:
                pair_locs.append(i)
        last_ch = ch
    return pair_locs

def remove_pairs(strng, pair_list):
    if len(pair_list)==0:
        return strng
    lst = [strng[:pair_list[0]-1]]
    last_index = pair_list[0]
    try:
        for index in pair_list[1:]:
            lst.append(strng[last_index+1:index-1])
            last_index=index
    except IndexError:
        pass
    lst.append(strng[last_index+1:])
    return cat(lst)

def react_all(strng):
    last_string = strng
    newstring = ''
    while True:
        pairs = find_all_pairs(last_string)
        newstring = remove_pairs(last_string, pairs)
        if newstring==last_string:
            return len(newstring)
        last_string=newstring
        
line5 = input_lines(5)[0].strip()
react_all(line5)

In [ ]:
#day5 part 2
#try 26 different versions.  In each, remove all instances of upper and lower versions of a single letter,
#then react the new string
#find the length of the shortest new string
import string

def remove_all(strng, unit):
    return strng.replace(unit.lower(),'').replace(unit.upper(),'')

def find_shortest_fixed(strng):
    min_len = len(strng)
    for unit in string.ascii_lowercase:
        test_strng = remove_all(strng, unit)
        newl = react_all(test_strng)
        min_len = newl if newl < min_len else min_len
    return min_len

find_shortest_fixed(line5)

In [ ]:
#day6 part1
#Given a list of coordinates, find largest "area" of coordinates that are only closest to one point.

#If I draw a grid, the area defined by any point on the edge of the grid will be infinite, and therefore, doesn't count
#step 1:  Create a grid.  Grid boundaries will be the min/max of the x and y values
coordinates = input_lines(6)
def nearest_point(point, coords):
    min_dist, min_coord = 9999999, 0
    for i,coord in enumerate(coords):
        dist = manhattan_distance(point, coord)
        if dist<min_dist:
            min_dist, min_coord = dist, i+1
        elif dist == min_dist:
            min_coord = 0
    return min_coord

def create_matrix(input_lines):
    coords = [tuple(map(int, line.strip().split(','))) for line in input_lines]
    xs, ys = list(zip(*coords))
    #make the matrix smaller by subtracting off minx, miny
    minx, maxx, miny, maxy = min(xs), max(xs), min(ys), max(ys)
    newcoords = [(c[0]-minx, c[1]-miny) for c in coords]
    matrix = np.zeros((maxx-minx+1,maxy-miny+1), dtype=int)
    it = np.nditer(matrix, flags=['multi_index'], op_flags=['readwrite'])
    for x in it:
        x[...] = nearest_point(it.multi_index, newcoords)
    return matrix

def find_largest_area(matrix):
    #eliminate any area touching the edges...those are infinite
    infs = set()
    sh = matrix.shape
    it = np.nditer(matrix, flags=['multi_index'])
    for x in it:
        if it.multi_index[0]==0 or it.multi_index[0]==sh[0]-1 or it.multi_index[1]==0 or it.multi_index[1]==sh[1]-1:
            infs.add(int(x))
    max_area = 0
    for i in range(np.max(matrix)):
        if i+1 in infs:
            continue
        area = np.sum(matrix==i+1)
        if area >= max_area:
            max_area = area
    return max_area

mtx = create_matrix(coordinates)
find_largest_area(mtx)
    

In [ ]:
#Day6 part2
#Find size of area with sum of all manhattan distances less than 10000
def total_distance_lt_thresh(point, coords, thresh=10000):
    total_dist = 0
    for i,coord in enumerate(coords):
        total_dist += manhattan_distance(point, coord)
    return total_dist <thresh
def create_matrix2(input_lines):
    coords = [tuple(map(int, line.strip().split(','))) for line in input_lines]
    xs,ys = list(zip(*coords))
    maxx, maxy = max(xs), max(ys)
    matrix = np.zeros((maxx+1, maxy+1), dtype=bool)
    it = np.nditer(matrix, flags=['multi_index'], op_flags=['readwrite'])
    for x in it:
        x[...] = total_distance_lt_thresh(it.multi_index, coords)
    return matrix
np.sum(create_matrix2(coordinates))

In [ ]:
#day7 part1
#Nearly identical to Euler problem 79
def process_instruction(ins):
    _, first, *_, second, _, _ = ins.split()
    return first, second

def make_graph(instruction_set):
    graph = defaultdict(set)
    for first, second in map(process_instruction, instruction_set):
        graph[second].add(first)
    return graph
    
def get_avail_moves(graph):
    return {x for y in graph.values() for x in y} - {x for x in graph.keys()}

def remove_last_move_from_priors(graph, move):
    #WooHoo, side effects!!!
    new_avail_moves = set()
    for new_move, priors in graph.items():
        graph[new_move]=priors-set(move)
        if len(graph[new_move])==0:
            new_avail_moves.add(new_move)
    for key in new_avail_moves:
        if key in graph:
            graph.pop(key)
    return new_avail_moves

def sort_instructions(instruction_set):
    graph=make_graph(instruction_set)
    avail = get_avail_moves(graph)
    last_move_made = sorted(list(avail))[0]
    moves=[last_move_made]
    while True:
        avail.remove(last_move_made)
        avail=avail.union(remove_last_move_from_priors(graph, last_move_made))
        try: last_move_made = sorted(list(avail))[0]
        except IndexError: break
        moves.append(last_move_made)
    return cat(moves)

sort_instructions(input_lines(7))

In [ ]:
#day7 Part2
import string
times = {x:i+61 for i,x in enumerate(string.ascii_uppercase)}

class Elf:
    def __init__(self, id):
        self.id, self.task, self.available, self.time_left = id, None, True, 0
    
    def take_task(self,task):
        self.task, self.available, self.time_left = task, False, times[task]
        
    def time_step(self):
        retval = None
        if not self.available:
            self.time_left -= 1
            if self.time_left == 0:
                self.available = True
                retval = self.task
                self.task = None
        return retval
    
def distribute_instructions(instruction_set, n=5):
    graph = make_graph(instruction_set)
    avail = get_avail_moves(graph)
    moves_completed = []
    elves = [Elf(i) for i in range(n)]
    time = 0
    while True:
        #First, let any elf that can take a task do so
        for elf in elves:
            if elf.available and len(avail) >0:
                task, *avail = sorted(list(avail))
                elf.take_task(task)
        #Then advance time by one second
        #If tasks are returned, they are completed
        completed = {elf.time_step() for elf in elves}
        completed.remove(None)
        moves_completed += sorted(list(completed))
        #take the completed tasks and remove the appropriate priors
        #remove_last_move_from_priors should also work with a set of moves
        avail = set(avail).union(remove_last_move_from_priors(graph, completed))
        time += 1
        if len(avail) == 0 and np.sum([elf.available for elf in elves])==len(elves):
            break
    return time, cat(moves_completed)

distribute_instructions(input_lines(7))

In [ ]:
#day8 part1

def recursive_metadata_sum(iterator):
    metadata_sum, n_child, n_meta = 0, next(iterator), next(iterator)
    metadata_sum+=sum((recursive_metadata_sum(iterator) for i in range(n_child)))
    metadata_sum += sum((next(iterator) for j in range(n_meta)))
    return metadata_sum

problem_nums = map(int, input_lines(8)[0].strip().split())
recursive_metadata_sum(problem_nums)

In [ ]:
#day8 part2

def get_child_val(vals, index):
    try:
        return vals[index]
    except IndexError:
        return 0

def recursive_node_value(iterator):
    n_child, n_meta = next(iterator), next(iterator)
    child_values = [recursive_node_value(iterator) for i in range(n_child)]
    if n_child==0:
        return sum((next(iterator) for j in range(n_meta)))
    return sum((get_child_val(child_values, next(iterator)-1) for j in range(n_meta)))

recursive_node_value(map(int, input_lines(8)[0].strip().split()))

In [ ]:
#day9 part1
class Ball:
    def __init__(self, idnum):
        self.id, self.cw, self.ccw = idnum, self, self
        
    def turn(self, current):
        if self.id % 23:
            self.cw, self.ccw, self.cw.ccw, self.ccw.cw = current.cw.cw, current.cw, self, self
            return 0, self
        else:
            removed = current.ccw.ccw.ccw.ccw.ccw.ccw.ccw
            current, removed.cw.ccw, removed.ccw.cw = removed.cw, removed.ccw, removed.cw
            return removed.id+self.id, current

def play_game(N_players, highest_ball):
    player_ids = np.arange(N_players, dtype=int)
    player_scores = np.zeros(N_players, dtype=int)
    for ballnum, player_id in enumerate(cycle(player_ids)):
        if ballnum == 0:
            current = Ball(ballnum)
            score, current = current.turn(current)
        else:
            score, current = Ball(ballnum).turn(current)
        player_scores[player_id] += score
        if ballnum==highest_ball:
            break
    return np.max(player_scores)

def process_input():
    n_players, *_, highest_ball, _ = Input(9).read().split()
    return int(n_players), int(highest_ball)

play_game(*process_input())

In [ ]:
#day9 part2
def part_2_input():
    n_players, *_, highest_ball, _ = Input(9).read().split()
    return int(n_players), int(highest_ball)*100

play_game(*part_2_input())

In [ ]:
#day10 part1

def process_line(line):
    _,x,y,_,vx,vy = line.strip().replace('<',' ').replace('>',' ').replace(',',' ').split()
    return int(x),int(y),int(vx),int(vy)

def advance_time(x,y,vx,vy, n):
    return x+n*vx, y+n*vy

def bbox_area(x,y):
    return (np.max(x)-np.min(x))*(np.max(y)-np.min(y))

def draw_array(x,y):
    minx, miny = np.min(x), np.min(y)
    arry = np.zeros((np.max(x)-np.min(x)+1, np.max(y)-np.min(y)+1), dtype=bool)
    for px,py in zip(x,y):
        arry[px-minx, py-miny] = True
    for yline in arry.T:
        for xval in yline:
            if xval: print('#', end='')
            else: print('.',end='')
        print()
                

#Find the total size of the bounding box of stars at each time step
#Operating on the assumption that the message will occur at the time
#Of minimum area of the bounding box
def find_minimum_bounding_box_time(pos):
    x0,y0,vx,vy = map(np.array, zip(*pos))
    bbox_size0 = bbox_area(x0,y0)
    time=0
    min_bbox = bbox_size0
    while True:
        time += 1
        nx, ny = advance_time(x0,y0,vx,vy,time)
        bbox = bbox_area(nx,ny)
        if bbox < min_bbox:
            min_bbox = bbox
            continue
        else:
            draw_array(*advance_time(x0,y0,vx,vy,time-1))
            return time-1, min_bbox
        
    

pos = [process_line(line) for line in input_lines(10)]
find_minimum_bounding_box_time(pos)
